In [1]:
!pip install transformers
!pip install question_generation
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 81.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 23.7 MB/s 
     |████████████████████████████████| 212 kB 92.9 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 83.4 MB/s 
     |████████████████████████████████| 140 kB 77.7 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |███████████████

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("algolet/mt5-base-chinese-qg")

model = AutoModelForSeq2SeqLM.from_pretrained("algolet/mt5-base-chinese-qg")

from question_generation import pipeline

# gpu版本
qg = pipeline("question-generation", device="cuda") 
qa = pipeline("question-answering", device="cuda") 

Downloading:   0%|          | 0.00/428 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.93M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

In [3]:
import pandas as pd
import glob
import json
import csv
import re
import os

path = r'./drive/MyDrive/Database/test_data/20220609/add_cols' # use your path

all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    test_list = []
    # 提取需要生成问题的文件名
    fname = filename.split('/')[-1:][0]
    fname = os.path.splitext(fname)[0]
    # 遍历词条信息进行问题生成
    with open(filename, 'r', encoding='utf-8-sig') as csvfile:
        csvReader = csv.DictReader(csvfile)
        i = 0
        for row in csvReader:
            y = {
                '_id':'',
                'text':'',
                'qa_pair':[]}
            i = i + 1
            text1 = row['实体']
            text2 = row['属性']
            text3 = row['值']
            full_text = text1 + ', '+ text2 +  ', '+  text3
            # 正则表达式，去除[]、()中的附加文本内容，减少对模型生成问题的无关影响
            full_text = re.sub("[\(\[].*?[\)\]]", "", full_text)
            # 对row进行转换，把原本的OrderedDict转换成string，此处因为保存原格式模型的表现更好，故此处进行保留
            # row = str(dict(row))
            print('FullText: ', full_text)
            y['_id'] = i
            questions = qg(full_text)
            j = 0
            for q in questions:
                z = {
                     'c_id':'',
                     'q':'',
                     'a':'',
                     'label':''
                }
                y['text'] = full_text
                answer = qa({
                    'question': str(q),
                    'context': full_text,
                })
                print('Q: ', q)
                # print('Row: ', row)
                if bool(answer) is False:
                    print('{ }')
                else:
                    j = j + 1
                    z['c_id'] = j
                    z['q'] = q
                    z['a'] = answer['answer']
                    print('Answer: ', answer['answer'])
                    print('\n')
                    y['qa_pair'].append(z)
            # 在满足q_a pair不空的情况下才进行添加
            if len(y['qa_pair']) == 0:
                pass
            else:
                test_list.append(y)
    # 对单个文件生成完后进行添加操作
    with open('./drive/MyDrive/Database/qa_results/20220610/'+fname+'.json', 'a', encoding='utf-8-sig') as file:
        json.dump(test_list, file, ensure_ascii=False) 

Streaming output truncated to the last 5000 lines.

FullText:  袁术, 描述, 袁术，字公路，汝南汝阳人。
Q:  袁术的字是什么?
Answer:  公路


Q:  袁术字什么?
Answer:  公路


Q:  谁是袁术的祖先?
{ }
FullText:  袁术, 别名, 公路
Q:  袁术的别名是什么?
{ }
Q:  袁术别名叫什么?
{ }
Q:  “袁术”别名叫做什么?
{ }
FullText:  袁术, 主要成就, 与孙坚共破董卓，称雄淮南
Q:  袁术击败了谁?
Answer:  董卓


Q:  袁术打败了谁
Answer:  董卓


Q:  董卓打败谁
Answer:  袁术


FullText:  袁术, 重要事件, 建安二年（197）称帝，建号仲氏
Q:  袁术是什么时候被称为袁术的?
{ }
Q:  袁术的谥号是什么?
{ }
Q:  建安二年袁术被称为什么?
Answer:  仲氏


FullText:  袁术, 时代, 东汉
Q:  袁术的时代
Answer:  东汉


Q:  袁术时代
Answer:  东汉


Q:  谁是袁术时代?
Answer:  东汉


FullText:  公孙瓒, 描述, 公孙瓒（？—199年），字伯珪，辽西令支（治今河北迁安市）人，出身贵族。
Q:  公孙瓒的字是什么?
Answer:  伯珪


Q:  伯珪是哪里人?
Answer:  辽西


Q:  公孙闿的籍贯是哪里?
{ }
FullText:  公孙瓒, 别称, 伯珪
Q:  公孙瓒的别名是什么?
Answer:  伯珪


Q:  伯祖瓒的别称是什么?
Answer:  公孙瓒, 别称, 伯珪


Q:  “伯祖瓒”的别名叫什么?
Answer:  公孙瓒


FullText:  公孙瓒, 主要成就, 驱叛胡于塞表、破黄巾于孟津
Q:  公孙瓒的成就是什么?
Answer:  驱叛胡于塞表、破黄巾于孟津


Q:  谁是公孙瓒的主要成就?
Answer:  驱叛胡于塞表、破黄巾于孟津


Q:  是谁打败了公孙瓒?
{ }
FullText:  糜芳, 描述, 糜芳是糜竺之弟，跟随刘备多年，曹操曾经上表他为彭城相，他辞官不受，继续追随刘备转战四方。
Q:  